### Import data

In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [3]:
#Load the data
#name='path of file'
#with_info= provides a tuple containing info about version, features and # samples of the data set
#as_supervised = True, loads the data in a 2-tuple structure [input,target]
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

2024-03-31 22:17:32.534060: W tensorflow/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata.google.internal".


Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /Users/kyotun/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [4]:
#extracting data
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

In [5]:
#validation comes from train part, there is no specific part for validation
#train contains 60,000 examples
#test contains 10,000 examples
#define the number of validation samples
#but we don't know if this num is integer or float
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples

In [6]:
#therefore, we convert the variable into a given data type
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

In [7]:
#store the num of test samples
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

In [8]:
#normally we'd like to scale the datain some way to make the result more numerically stable (inputs between 0-1)
#take an MNIST iamge and its label
#0-255
#we want float
#you can scale your data in other ways if you see fit.
#make sure that func. takes image and label
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label


In [9]:
#scale the whole train dataset and store it in variable
scaled_train_and_validation_data = mnist_train.map(scale)